In [ ]:
import pandas as pd
import numpy as mp
import datetime
import matplotlib.pyplot as plt
file = 'hotspotResult425.csv'
df = pd.read_csv(file)

In [ ]:
#dealt with old version of csv
import pandas as pd
from datetime import datetime


file = 'precisefinal.csv'
df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)
for i in df:
    print(i)
# file = 'precisefinal.csv'
# df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)
# df['clickTime'] = df['clickTime'].apply(lambda x: process(x))
df['clickTime'] = pd.to_datetime(df['clickTime'], format='%Y-%m-%d %H:%M:%S')


df['hs_name'] = df['hotspotName'].apply(lambda x: x.split('_')[0])
print(df['hs_name'])
hs_names = df['hs_name'].unique()

for name in hs_names:
    if name.startswith('p'):
        hs_types = ['quotes', 'vocabulary']
        for hs_type in hs_types:
            tmp = df.loc[(df['hs_name'] == name) & (df['hotspotType'] == hs_type)].copy()
            print(tmp.shape[0])
            if tmp.shape[0] > 1:
                tmp['diff'] = tmp['clickTime'].diff(1)
                tmp['diff'] = tmp['diff'].apply(lambda x: x.total_seconds())
                indexes = tmp.loc[(tmp['diff'] >= 0) & (tmp['diff'] <= 2)].index
                print(indexes)
                df = df.loc[~df.index.isin(indexes)]
                
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,23,29,
          30,31,43,45,50,56,57,59,61,64,65,71,72,73,79,81,
          83,90,97,100,102,108,114,115,116,121,125,126,
          128,13,131,132,134,136,138,140,143,145,146,147,
          150,152,154,158,159,160,161,163,164,
          167,168,174,184,199,200,201,209,213,215,217,
          239,243,251,253,255,257,259,265,267]:
    
    indexNames = df[ df['username'] == i ].index
    # Delete row indexes from dataFrame
    df.drop(indexNames , inplace=True)

df = df.sort_values(by=['username', 'opentime'])
df.to_csv('o.csv', index=False)




In [ ]:
df

import pandas as pd


def process(time):
    time = time.split()
    time[0] = time[0].split('/')
    time[0][2] = '20' + time[0][2]
    time[0] = '/'.join(time[0])
    time = ' '.join(time)
    return time

def duration(x):
    start_time = x.iloc[0, 3]
    type_ = x.iloc[0, 1]
    hotspot_name = x.iloc[:, 2]
    check = ''
    if 'p17_nextButton' in hotspot_name.tolist() or 'q17_nextButton' in hotspot_name.tolist():
        check = 'yes'
    else:
        check = 'no'

    q_dura = 0
    p_dura = 0
    end_time = start_time
    for i in range(1, x.shape[0]):
        click, name = x.iloc[i, 3], x.iloc[i, 1]
        if name == type_:
            end_time = click
        else:
            if type_ == 'q':
                q_dura =round(q_dura+((end_time - start_time).total_seconds())/60, 2)
            else:
                p_dura =round(p_dura+((end_time - start_time).total_seconds())/60, 2)
            start_time = click
            end_time = click
            type_ = name
    if type_ == 'q':
        q_dura =round(q_dura+((end_time - start_time).total_seconds())/60, 2)
    else:
        p_dura =round(p_dura+((end_time - start_time).total_seconds())/60, 2)
    return q_dura, p_dura, check


file = 'newwww_changed.csv'
df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)

# df['clickTime'] = df['clickTime'].apply(lambda x: process(x))
df['clickTime'] = pd.to_datetime(df['clickTime'], format='%Y/%m/%d %H:%M:%S')

df['date'] = df['clickTime'].apply(lambda x: x.strftime('%Y/%m/%d'))

df['name'] = df['hotspotName'].apply(lambda x: x[0])

df = df[['username', 'name', 'hotspotName', 'clickTime', 'date', 'bookid']]

df = df.drop_duplicates()
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,23,29,30,31,43,45,50,56,57,59,61,64,65,71,72,73,79,81,
          83,90,97,100,102,108,114,115,116,121,125,126,
          128,13,131,132,134,136,138,140,143,145,146,147,
          150,152,154,158,159,160,161,163,164,
          167,168,174,184,199,200,201,209,213,215,217,
          239,243,251,253,255,257,259,265,267]:
          
    indexNames = df[ df['username'] == i ].index
    # Delete row indexes from dataFrame
    df.drop(indexNames , inplace=True)
book_id = []
username = []
q_duration = []
p_duration = []
dates = []
checks = []

for user in df['username'].unique():
    unique_dates = df.loc[df['username'] == user]['date'].unique()
    for date in unique_dates:
        tmp = df.loc[(df['username'] == user) & (df['date'] == date)]
        q, p, check = duration(tmp)
        username.append(user)
        q_duration.append(q)
        p_duration.append(p)
        dates.append(date)
        checks.append(check)
        book_id.append(tmp['bookid'].iloc[0])

result = pd.DataFrame({'username': username, 'Q Clicktime duration': q_duration,
                       'P Clicktime duration': p_duration, 'Date': dates, 'Bookid': book_id, 'Check': checks})
result.to_csv('qwe.csv', index=False)
#from google.colab import files
#uploaded = files.upload()
#import io
#df1=pd.read_csv(io.BytesIO(uploaded['hotspotResult425.csv']))

In [ ]:

df = pd.read_csv('qwe.csv')

participants = []
r1q = []
r1p = []
r1 = []
r2q = []
r2p = []
r2 = []
for user in df['username'].unique():
    tmp = df.loc[df['username'] == user]
    participants.append(user)
    bookids = tmp['Bookid'].unique()
    
    if bookids.shape[0] == 1:
#         print(bookids.shape[0])
        if bookids[0] == 0:
            r2q.append(0)
            r2p.append(0)
            r2.append(0)
        else:
            r1q.append(0)
            r1p.append(0)
            r1.append(0)

    for bookid in bookids:
        if bookid == 0:
            q_durations = tmp.loc[tmp['Bookid'] == bookid]['Q Clicktime duration'].sum()
            p_durations = tmp.loc[tmp['Bookid'] == bookid]['P Clicktime duration'].sum()
            r1q.append(q_durations)
            r1p.append(p_durations)
            r1.append(round(q_durations + p_durations,2))
        else:
            q_durations = tmp.loc[tmp['Bookid'] == bookid]['Q Clicktime duration'].sum()
            p_durations = tmp.loc[tmp['Bookid'] == bookid]['P Clicktime duration'].sum()
            r2q.append(q_durations)
            r2p.append(p_durations)
            r2.append(round(q_durations + p_durations,2))

res = pd.DataFrame({'Participant_ID': participants, 'R1Q_READ': r1q, 'R1P_READ': r1p, 'R1_READ': r1,
                    'R2Q_READ': r2q, 'R2P_READ': r2p, 'R2_READ': r2})
res['T_READ'] = round(res['R1_READ'] + res['R2_READ'],2)
res.to_csv('lit.csv', index=False)

In [ ]:
df = pd.read_csv('newwww_changed.csv')
df2 = pd.read_csv('newwww_changed.csv')
df2.drop(df.index, inplace=True)

indexDate = parser.parse("Aug 28 1980")
indexUser = 9999999999
for row in df.iterrows():
    date = parser.parse(row[1][2]).date()
    if date != indexDate and row[1][1] != indexUser:
#         print(indexUser)
#         print(indexDate)
#         print(row[1][1])
#         print(date)
        indexUser = row[1][1]
        indexDate = date
        df2 = df2.append(row[1])
    elif date == indexDate and row[1][1] == indexUser:
        df2 = df2.append(row[1])

df2.to_csv("sss.csv",index=False)


with open('sss.csv', 'r') as t1, open('newwww_changed.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('update.csv', 'w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)

Correctness+Question Answered 

In [ ]:

#必要的package
import pandas as pd
import numpy as mp
import datetime
import matplotlib.pyplot as plt

In [ ]:
#导入文件
import pandas as pd

df1 = pd.read_csv ("/Users/wangyiting/Desktop/new_cleaned.csv")


In [ ]:
#拿出userId
userList = []
for i in df1["username"]:
    if i not in userList:
        userList.append(i)

In [ ]:
#新建两个空的dic, dic1用来放每个学生 实际做了几个选择题的数量， dic2用来放每个学生 第一次就选择了正确的答案的数量
dic_1={}
dic_2={}


for i in userList:
    dic_1[i]=0
    dic_2[i]=0
    #把每个学生所有的数据分别拿出来
    student0 = df1[df1.username == int(i)]
    #新建空的list2用于后面放题号
    list2=[]
    #新建count_right 用于后面计算第一次就选择了正确答案的数量
    count_right=0
    for index, row in student0.iterrows():
        #把每一行拿出来
        a=row
        #如果是discussion类型，就代表是多选题
        if a.hotspotType=="discussion":
            #name 是题号
            name=a.hotspotName
            name=str(name)[1:]
            #如果题号没有被记录
            if name not in list2:
                #把题号放进list
                list2.append(name)
                #看第一遍result 是否是对的，对的就次数加一
                result=str(a.clickResult)
                if result.endswith("answer"):
                    count_right+=1
    dic_1[i]=len(list2)
    dic_2[i]=count_right

In [ ]:
#算正确率， 保留三位小数
dic_rate={}
for i in userList:
    i=int(i)
    #判断 实际选择题次数是否为0， 为0的话直接算0
    if dic_1[i]==0:
        dic_rate[i]=0
    else:
        #用正确率/次数， 保留三位小数
        dic_rate[i]=round(dic_2[i]/dic_1[i],3)

In [ ]:
#a是userID, b是每个学生实际做了几个选择题的数量， c是每个学生第一次就选择了正确的答案的数量, d是c/b-Right Rate
a=userList
b=list(dic_1.values())
c=list(dic_2.values())
d=list(dic_rate.values())

In [ ]:
#改成dataframe格式
df3 = pd.DataFrame({'UserName' : a,'NumQuestionAnswer':b, 'NumRightFirstly' : c,'RightRate':d})

In [ ]:
#导出文件
df3.to_csv('RightRate.csv')